# Fetching Recent Season Matches

Example notebook demonstrating how to extract matches from the last two weeks for a specific championship.

### Overview

This notebook demonstrates:
- Navigating the API hierarchy (Countries → Championships → Seasons)
- Efficiently fetching recent matches using reverse pagination
- Filtering by date, season, and match status
- Converting API responses to pandas DataFrame

**Use case**: Get all completed Premier League matches from the last 14 days.

In [22]:
import math
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
from soccer_info import quick_client
from IPython.display import JSON

### Constants

In [23]:
# Number of days back to fetch matches from
DAYS_BACK = 14

# The API occasionally returns incorrect pagination metadata on page 1
# (e.g., item count may not reflect actual data). We query a stable page (page 3)
# to get accurate pagination information for calculating total pages.
FIRST_STABLE_PAGE = 3

# Limit number of match pages to check backwards from latest
MAX_PAGES_TO_CHECK = 5

### Setup Client

In [24]:
load_dotenv()
client = quick_client()

### Browse Countries

In [25]:
countries = client.countries.get_list()

In [26]:
JSON(countries.model_dump())

<IPython.core.display.JSON object>

### Select Country

In [27]:
gb = next(
    item for item in countries.result if item.name == "United Kingdom"
)
JSON(gb.model_dump())

<IPython.core.display.JSON object>

### Browse Championships

In [28]:
gb_championships = client.championships.get_list(country=gb.code)
JSON(gb_championships.model_dump())

<IPython.core.display.JSON object>

### Select Championship

In [29]:
primer_league = next(
    item for item in gb_championships.result if item.name == "England Premier League"
)
JSON(primer_league.model_dump())

<IPython.core.display.JSON object>

### Get last season name

In [30]:
primer_league_details = client.championships.get_by_id(championship_id=primer_league.id)

In [31]:
last_season = primer_league_details.result[0].seasons[-1]
JSON(last_season.model_dump())

<IPython.core.display.JSON object>

### Estimate Championship's pagination

In [32]:
matches = client.matches.get_by_filter_basic(championship_id=primer_league.id, page=FIRST_STABLE_PAGE)

In [33]:
pagination = matches.pagination[0]
pages = math.ceil(pagination.items / pagination.per_page)
pages

147

### Get championship's last season games from the last week

In [34]:
filtered_matches = []
continue_scan = True
cut_off_date = datetime.today() - timedelta(days=DAYS_BACK)
i = 0
df = None
while continue_scan:
    print(f"Getting page {i}")
    matches_page = client.matches.get_by_filter_basic(championship_id=primer_league.id, page=pages-i)
    df_page = pd.DataFrame([
        {
            'date': match.date,
            'team_a': match.teamA.name,
            'a_score': match.teamA.score.f,
            'team_b': match.teamB.name,
            'b_score': match.teamB.score.f,
            'stadium': match.stadium.name
         }
        for match in matches_page.result
        if match.championship.s_name == last_season.name
           and match.status == "ENDED"
    ])
    df_page['date'] = pd.to_datetime(df_page['date'])
    if not df_page.empty:
        df = df_page if df is None else pd.concat([df, df_page])
        if df_page['date'].min() < cut_off_date or i >= MAX_PAGES_TO_CHECK:
            continue_scan = False
        else:
            i += 1

Getting page 0
Getting page 1


In [35]:
df = df[df['date'] >= cut_off_date]
df

,date,team_a,a_score,team_b,b_score,stadium
0,2026-01-17 15:00:00,Sunderland,2,Crystal Palace,1,Stadium of Light
1,2026-01-17 15:00:00,Liverpool,1,Burnley,1,Anfield
2,2026-01-17 15:00:00,Leeds,1,Fulham,0,Elland Road
3,2026-01-17 17:30:00,Nottm Forest,0,Arsenal,0,City Ground
6,2026-01-04 12:30:00,Leeds,1,Man Utd,1,Elland Road
7,2026-01-04 15:00:00,Tottenham,1,Sunderland,1,Tottenham Hotspur Stadium
8,2026-01-04 15:00:00,Newcastle,2,Crystal Palace,0,St James' Park
9,2026-01-04 15:00:00,Everton,2,Brentford,4,Everton Stadium
10,2026-01-04 15:15:00,Fulham,2,Liverpool,2,Craven Cottage
11,2026-01-04 17:30:00,Man City,1,Chelsea,1,Etihad Stadium
